In [5]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [30]:
from importlib import reload
import json
import utils

reload(utils)

chat = utils.chat

In [31]:
query_rewrite_prompt = """
    You are an expert in language and understanding semantic meaning.
    You read input from person and your job is to expand the input
    into stand alone atomic questions.

    You must return JSON using the template below.

    Example:
    Input: Who is Mick Jagger and how old is he?
    Output: 
    {
        "questions": [
            "Who is Mick Jagger?",
            "How old is Mick Jagger?"
        ]
    }
"""

def query_rewrite(input: str) -> list[str]:
        
        messages = [
            {"role": "system", "content": query_rewrite_prompt},
            {"role": "user", "content": f"The user question to rewrite: '{input}'"},
        ]
        config = {
            "response_format": {"type": "json_object"},
        }
        output = chat(messages, model = "gpt-3.5-turbo-0125", config=config, )
        try:
            return json.loads(output)["questions"]
        except json.JSONDecodeError:
            print("Error decoding JSON")
        return []


In [34]:
response = query_rewrite("Who directed the movie 'The Godfather', how long is it and what is it about?")
print(f"Query rewrite results: {response}")

Query rewrite results: ["Who directed the movie 'The Godfather'?", "How long is the movie 'The Godfather'?", "What is the movie 'The Godfather' about?"]


In [ ]:
query_update_prompt = """
    You are an expert at updating questions to make the them more atomic, specific and easier to find the answer for.
    You do this by filling in missing information in the question, with the extra information provided to you in previous answers. 
    
    You respond with the updated question that has all information in it.
    Only edit the question if needed. If the original question already is atomic, specific and easy to answer, you keep the original.
    Do not ask for more information than the original question. Only rephrase the question to make it more complete.
    
    JSON template to use:
    {
        "question": "question1"
    }
"""

def query_update(input: str, answers: list[any]) -> str: 
    messages = [
        {"role": "system", "content": query_update_prompt},
        *answers,
        {"role": "user", "content": f"The user question to rewrite: '{input}'"},
    ]
    config = {"response_format": {"type": "json_object"}}
    output = chat(messages, model = "gpt-3.5-turbo-0125", config=config, )
    try:
        return json.loads(output.choices[0].message.content)["question"]
    except json.JSONDecodeError:
        print("Error decoding JSON")
    return []

In [ ]:

tool_picker_prompt = """
    Your job is to chose the right tool needed to respond to the user question. 
    The available tools are provided to you in the prompt.
    Make sure to pass the right and the complete arguments to the chosen tool.
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "capital_by_country",
            "description": "Get the capital of a country by providing the country name",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The country name",
                    }
                },
                "required": ["country"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "country_by_capital",
            "description": "Get the country name by providing the name of the capital city",
            "parameters": {
                "type": "object",
                "properties": {
                    "capital": {
                        "type": "string",
                        "description": "The capital city name",
                    }
                },
                "required": ["capital"],
            },
        },
    }
]

def handle_tool_calls(tools: dict[str, any], llm_tool_calls: list[dict[str, any]]):
    output = []
    if llm_tool_calls:
        for tool_call in llm_tool_calls:
            function_to_call = tools[tool_call.function.name]
            function_args = json.loads(tool_call.function.arguments)
            res = function_to_call(**function_args)
            output.append(res)
    return output



In [ ]:
def route_question(question: str, tools: dict[str, any], answers: list[dict[str, str]]):
    llm_response = chat(
        [
            {
                "role": "system",
                "content": tool_picker_prompt,
            },
            *answers,
            {
                "role": "user",
                "content": f"The user question to find a tool to answer: '{question}'",
            },
        ],
        model = "gpt-3.5-turbo-0125",
        tools=tools,
    )
    tools_choices = llm_response.choices[0].message
    llm_tool_calls = tools_choices["tool_calls"]
    return handle_tool_calls(tools, llm_tool_calls)

def handle_user_input(input: str, answers: list[dict[str, str]] = []):
    atomic_questions = query_rewrite(input)
    for question in atomic_questions:
        updated_question = query_update(question, answers)
        response  = route_question(updated_question, tools, answers)
        answers.append({"role": "assistant", "content": f"For the question: '{updated_question}', we have the answer: '{json.dumps(response)}'"})
    return answers

In [ ]:
answer_critique_prompt = """
    You are an expert at identifying if questions has been fully answered or if there is an opportunity to enrich the answer.
    The user will provide a question, and you will scan through the provided information to see if the question is answered.
    If anything is missing from the answer, you will provide a set of new questions that can be asked to gather the missing information.
    All new questions must be complete, atomic and specific.
    However, if the provided information is enough to answer the original question, you will respond with an empty list.

    JSON template to use for finding missing information:
    {
        "questions": ["question1", "question2"]
    }
"""

def critique_answers(question: str, answers: list[dict[str, str]]) -> list[str]:
    messages = [
        {
            "role": "system",
            "content": answer_critique_prompt,
        },
        *answers,
        {
            "role": "user",
            "content": f"The original user question to answer: {question}",
        },
    ]
    config = {"response_format": {"type": "json_object"}}
    output = chat(messages, model="gpt-3.5-turbo-0125", config=config)
    try:
        return json.loads(output.choices[0].message.content)["questions"]
    except json.JSONDecodeError:
        print("Error decoding JSON")
    return []

In [ ]:
main_prompt = """
    Your job is to help the user with their questions.
    You will receive user questions and information needed to answer the questions
    If the information is missing to answer part of or the whole question, you will say that the information 
    is missing. You will only use the information provided to you in the prompt to answer the questions.
    You are not allowed to make anything up or use external information.
"""

def main(input: str):
    answers = handle_user_input(input)
    critique = critique_answers(input, answers)

    if critique:
        answers = handle_user_input(" ".join(critique), answers)

    llm_response = chat(
        [
            {"role": "system", "content": main_prompt},
            *answers,
            {"role": "user", "content": f"The user question to answer: {input}"},
        ],
        model="gpt-4-turbo",
    )

    return llm_response